In [1]:
import os
import sys
import matplotlib as mpl
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
from Bio import SeqIO
from Bio import Phylo
import pandas as pd
import seaborn as sns
import pysam
mpl.rcParams['pdf.fonttype'] = 42
%matplotlib inline

In [2]:
def checkSlash(directory):
    if directory[-1] != '/':
        directory = directory + '/'
    return directory

In [3]:
unmappedDir = '12_bwa_unmapped/'

In [5]:
#IF a read maps to some but not others...
#WHICH of the unmapped reads are not shared by all?
#DO any of them cluster?
readDir = '13_bwa_horizontal/'
if not os.path.exists(readDir):
    os.makedirs(readDir)
for subdirs, dirs, files in os.walk(unmappedDir):
    for d in dirs:
        ####
        ####PREPARE DICTIONARY OF READS
        subDict = {}
        for f in os.listdir(unmappedDir + d):
            if 'contig' not in f:
                continue
            contig = f.split('_')[1]
            mapped = f.split('_')[3]            
            inFile = open(unmappedDir + d + '/' + f, 'r')
            unmappedDict = {}
            for line in inFile:
                line = line.strip().split('\t')
                readID = line[0]
                unmappedDict[readID] = line
            inFile.close()
            if mapped in subDict:
                subDict[mapped].append([f, unmappedDict])
            else:
                subDict[mapped] = [[f, unmappedDict]]
        ####
        ####IDENTIFY READS THAT DON'T MAP TO ANY SAMPLE
        badReads = {}
        for i, mapped in enumerate(subDict):
            unmappedSet = set()
            for j, result in enumerate(subDict[mapped]):
                f = result[0]
                unmappedDict = result[1]
                if j == 0:
                    unmappedSet = set(unmappedDict.keys())
                else:
                    unmappedSet = unmappedSet.intersection(unmappedDict.keys())
            #print(mapped + ': ' + str(len(unmappedSet)))
            badReads[mapped] = unmappedSet
        ####
        ####RECORD READS THAT MAP TO SOME BUT NOT OTHERS
        ####USE DIAMOND AGAINST NR?!?!
        for i, mapped in enumerate(subDict):
            readsOfInterest = {}
            for j, result in enumerate(subDict[mapped]):
                f = result[0]
                unmappedDict = result[1]
                ct = 0
                for read in unmappedDict:
                    if read not in badReads[mapped]:
                        if read not in readsOfInterest:
                            seq = unmappedDict[read][9]
                            readsOfInterest[read] = seq
                        ct += 1
                #Create pandas dataframe w/readIDs maybe?
                #Or get locations and search for them in annotations?
                #Or just blast against nr?
                #print(f + ': ' + str(ct))
            if not os.path.exists(readDir + d):
                os.makedirs(readDir + d)
            outFileName = readDir + d + '/' + mapped + '_horizontal.fa'
            outFile = open(outFileName, 'w')
            for r in readsOfInterest:
                outFile.write('>' + r + '\n' + readsOfInterest[r] + '\n')
            outFile.close()
            os.system('blastn -task megablast -db nt_blast/nt -query ' + outFileName + ' -out ' + \
                      outFileName.replace('.fa', '.out'))
            #diamond blastx -d nr -q reads.fna -a reads -t <temporary directory>
            #('blastn megablast ')
            #os.system('diamond blastx -d nr_diamond/nr -q ' + outFileName + ' -a ' + \
            #          outFileName.replace('.fa', '') + ' -t tmp/')

In [ ]:
num_threads
outfmt